<a href="https://colab.research.google.com/github/pradyutsingh/Neural-network/blob/master/imdb_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [17]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [18]:
pip install wandb

In [19]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)



In [20]:
import wandb

In [21]:
wandb.login()

True

In [22]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [23]:
from wandb.tensorflow import WandbHook
wandb.init(project="imbd", sync_tensorboard=True)

W&B Run: https://app.wandb.ai/s_pradyut/imbd/runs/2xay1mk5

In [25]:
hyperparameter_defaults = dict(
  hidden_layer_size = 6,
  learn_rate = 0.01,
  epochs = 8,
)
wandb.init(config=hyperparameter_defaults)
config = wandb.config

In [26]:
adamopt = tf.keras.optimizers.Adam(config.learn_rate)

In [27]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type)

In [28]:
#now reversing the word indices
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])
def decode_rev(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_rev(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [29]:
model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(config.hidden_layer_size,activation='relu'),
          tf.keras.layers.Dense(1,activation='sigmoid'),
])

In [35]:
from wandb.keras import WandbCallback

In [36]:
model.compile(loss='binary_crossentropy',optimizer=adamopt,metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(padded,testing_labels_final,epochs = config.epochs
          ,validation_data = (testing_padded, testing_labels_final))

Epoch 1/8
782/782 [==============================] - 6s 7ms/step - loss: 0.2609 - accuracy: 0.8356 - val_loss: 1.6700 - val_accuracy: 0.4911
Epoch 2/8
782/782 [==============================] - 5s 7ms/step - loss: 0.2299 - accuracy: 0.8565 - val_loss: 1.9232 - val_accuracy: 0.4920
Epoch 3/8
782/782 [==============================] - 7s 9ms/step - loss: 0.2088 - accuracy: 0.8706 - val_loss: 2.0056 - val_accuracy: 0.4980
Epoch 4/8
782/782 [==============================] - 6s 7ms/step - loss: 0.1859 - accuracy: 0.8845 - val_loss: 2.2222 - val_accuracy: 0.4925
Epoch 5/8
782/782 [==============================] - 5s 7ms/step - loss: 0.1646 - accuracy: 0.8964 - val_loss: 2.4184 - val_accuracy: 0.4943
Epoch 6/8
782/782 [==============================] - 5s 7ms/step - loss: 0.1565 - accuracy: 0.9062 - val_loss: 2.4591 - val_accuracy: 0.4928
Epoch 7/8
782/782 [==============================] - 5s 7ms/step - loss: 0.1439 - accuracy: 0.9144 - val_loss: 2.6987 - val_accuracy: 0.4935
Epoch 8/8
782

Error in callback <function _init_jupyter.<locals>.cleanup at 0x7f13b83e06a8> (for post_run_cell):


KeyboardInterrupt: ignored